<a href="https://colab.research.google.com/github/Zishan-923212/colab-tools/blob/main/COLAB_TOOLS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##__Mount GDrive to /content/drive__
#####Use one of below methods. Mounting with Google Drive File Stream app is more comfortable, but it has some writing issues. So you can avoid them by using Rclone instead.

###__Mount with Google Drive File Stream__

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount GDrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive!
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

###__Mount with Rclone - (More Accurate) - (Updated)__
#####**- Multiple drives including team drives can be mounted at the same runtime session**

In [ ]:
#@markdown <br><center><img src='https://rclone.org/img/logo_on_light__horizontal_color.svg' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount GDrive to /content/drive_name</h3></center><br>
drive_name = "AniMov" #@param {type:"string"}
#@markdown If you want to configure this as a team drive, check the below checkbox
team_drive = True #@param {type:"boolean"}

import subprocess, time, json, os
from google.colab import output
from IPython.display import HTML, clear_output

def formatx(i, j, k):
    val=auth_response[i].split(": ")[1][1-j:-2+j+k]
    return val

rclone_status = subprocess.check_output("command -v rclone &> /dev/null",  shell=True).decode()
if rclone_status:
  pass
else:
  !curl https://rclone.org/install.sh | sudo bash
  !rclone config file
  !touch /root/.config/rclone/rclone.conf
  output.clear()
  display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Rclone: Installed successfully .</h2><br>"))
  time.sleep(1)
  display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Next: Drive authentication...</h2><br>"))
  time.sleep(2)
  output.clear()

client_id="202264815644.apps.googleusercontent.com"
client_secret="X4Z3ca8xfWDb1Voo-F9a7ZxJ"
print("Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id={}&redirect_uri=urn:ietf:wg:oauth:2.0:oob&scope=https://www.googleapis.com/auth/drive&response_type=code".format(client_id))
auth_code=input("Enter your authorization code: ")
auth_response = subprocess.check_output("curl -d client_id={} -d client_secret={} -d grant_type=authorization_code -d redirect_uri=urn:ietf:wg:oauth:2.0:oob -d code={} https://accounts.google.com/o/oauth2/token".format(client_id, client_secret, auth_code), shell=True).decode().split("\n")[1:6]
access_token, expires_in, refresh_token, scope, token_type = formatx(0, 0, 0), formatx(1, 1, 0), formatx(2, 0, 0), formatx(3, 0, 0), formatx(4, 0, 1)
token_in_json = "{"+", ".join("".join("".join(auth_response).split("  ")).split(","))+"}"

output.clear()
display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Authenticated successfully.</h2><br>"))
time.sleep(2)
output.clear()

if team_drive:
  print("Fetching team drive list...")
  check_drives = subprocess.check_output('''curl \
  'https://www.googleapis.com/drive/v3/drives' \
  --header 'Authorization: Bearer {}' \
  --header 'Accept: application/json' \
  --compressed'''.format(access_token),  shell=True).decode()
  drive_list = json.loads(check_drives).get('drives')
  print("Choose a number from below.(If you want to use the 1st team drive from the below list, type 1 and press enter)")
  for i in range(len(drive_list)):
    print(i+1,"/", drive_list[i].get('name'))
    print("  \\","\""+drive_list[i].get('id')+"\"")
  selected_num = int(input("Enter the number : "))
  root_id = drive_list[selected_num-1].get('id')
else:
  print("Fetching root id...")
  check_about = subprocess.check_output('''curl \
  'https://www.googleapis.com/drive/v2/about' \
  --header 'Authorization: Bearer {}' \
  --header 'Accept: application/json' \
  --compressed'''.format(access_token),  shell=True).decode()
  root_id = json.loads(check_about).get('rootFolderId')

#read file
rfile = open(r"/root/.config/rclone/rclone.conf", "r")
rfile_content = rfile.readlines()
rfile.close()
#write file
wfile = open(r"/root/.config/rclone/rclone.conf","a+")
if len(rfile_content) > 0:
  wfile.write("\n\n")
wfile.write("[{}]\ntype = drive\nscope = drive\ntoken = {}\n".format(drive_name, token_in_json))
if team_drive:
  wfile.write("team_drive = {}".format(root_id))
else:
  wfile.write("root_folder_id = {}".format(root_id))
wfile.close()

output.clear()
display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Configuration: Completed successfully</h2><br>"))
time.sleep(1)
display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Next: Mounting drive...</h2><br>"))
time.sleep(2)
output.clear()

rclone_config_name = drive_name
local_mount_location = "/content/"+drive_name+"/"

# clear nohup
open("nohup.out", 'w').close()

# unmount first
#!fusermount -u $local_mount_location 2>/dev/null

# mount without waiting for the command to complete
!mkdir $local_mount_location 2>/dev/null
!nohup rclone mount $rclone_config_name: $local_mount_location --buffer-size 96M &

# Show the output that was written to nohup
time.sleep(3)
f = open(r"nohup.out", "r")
nohupText = f.read()
f.close()

dirs = os.listdir(local_mount_location)

if len(dirs) > 0:
  clear_output(wait=True)
  print("Succeeded. ", str(len(dirs)), "dirs found at", local_mount_location)

else:
  print("\n\nNot succeeded. No files or directories in mounted location. \nCheck your config name and content. If the rclone command was not found, run the cell above.\n\n")
  print("log:\n", nohupText)

# Remote Upload

In [ ]:
#@title Direct Link
Storage = "/content/drive/MyDrive/" #@param {type:"string"}
#@markdown To generate filename from url, press enter in filename input.
#@markdown
#@markdown To stop input and begin downloads, press Enter in url input
import os
if not os.path.exists(Storage):
  os.mkdir(Storage)
%pip install wget
downloadurl=[]
filename=[]
%cd "$Storage"
while 1==1 :
    url=str(input("Enter url "))
    url.lower()
    if url == "":
        break
    name=str(input("Enter filename with format "))
    downloadurl.append(url)
    filename.append(name)
downloadurl.reverse()
filename.reverse()
while downloadurl != []:
    namefile=filename.pop()
    urldownload=downloadurl.pop()
    if namefile=="":
        !wget --no-check-certificate "$urldownload" "$Storage"
        pass
    !wget --no-check-certificate -O "$namefile" "$urldownload" "$Storage"

In [ ]:
#@title Torrent
Storage = "/content/drive/MyDrive/Games/" #@param {type:"string"}
!apt install python3-libtorrent
!python -m pip install lbry-libtorrent
import libtorrent as lt
import time
import datetime
import sys
import os
%cd "$Storage"
ses = lt.session()
ses.listen_on(6881, 6891)
params = {
    'save_path': "Torrent",
    'storage_mode': lt.storage_mode_t(2)
    }
link = str(input("Enter Magnet Link"))
print(link)

handle = lt.add_magnet_uri(ses, link, params)
ses.start_dht()

begin = time.time()
print(datetime.datetime.now())

print ('Downloading Metadata...')
while (not handle.has_metadata()):
    time.sleep(1)
print ('Got Metadata, Starting Torrent Download...')

print("Starting", handle.name())

while (handle.status().state != lt.torrent_status.seeding):
    s = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata', \
            'downloading', 'finished', 'seeding', 'allocating']
    sys.stdout.write("\r" + '%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' % \
            (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, \
            s.num_peers, state_str[s.state]))
    sys.stdout.flush()
    time.sleep(5)

end = time.time()
print(handle.name(), "COMPLETE")

print("Elapsed Time: ",int((end-begin)//60),"min :", int((end-begin)%60), "sec")

print(datetime.datetime.now())

##Pornhub Downloader

In [ ]:
#@title Install
from IPython.display import clear_output
import os
os.system('pip3 install youtube-dl')
os.system('pip3 install prettytable')
os.system('pip3 install bs4')
os.system('pip3 install requests')
%cd /content
os.system('git clone https://github.com/mariosemes/PornHub-downloader-python.git')
%cd PornHub-downloader-python
clear_output()
print("Install Finished")

<center><b><u>USES</u></b></center>
<table>
  <tr>
    <td><b>Tool</b></td>
    <td><b>command</b></td>
    <td><b>item</b></td>
  </tr>
  <tr>
    <td>python3 phdler.py</td>
    <td>start</td>
  </tr>
    <td>python3 phdler.py</td>
    <td>custom</td>
    <td>url | batch</td>
  </tr>
  </tr>
    <td>python3 phdler.py</td>
    <td>add</td>
    <td>model | pornstar | channel | user | playlist | batch</td>
  </tr>
  </tr>
    <td>python3 phdler.py</td>
    <td>list</td>
    <td>model | pornstar | channel | user | playlist | all</td>
  </tr>
  </tr>
    <td>python3 phdler.py</td>
    <td>delete</td>
    <td>model | pornstar | channel | user | playlist</td>
  </tr>

In [ ]:
#@title Start
#@markdown Starts downloading all new videos from database.
!python3 phdler.py start

In [ ]:
#@title Custom
item = "https://www.pornhub.com/view_video.php?viewkey=6559efddb63e7" #@param {type:"string"}
#@markdown To download a single video, paste Video URL.<br>To batch download a number of videos, paste the full Path to the .txt file containing each URL in separate lines.
!python3 phdler.py custom "$item"

In [ ]:
#@title Add
item = "" #@param {type:"string"}
#@markdown To download all videos from model/ pornstar/ channel/ user/ playlist.<br>Paste URL. This adds the URL to the database.<br>For batch download, paste the full Path to the .txt file containing each URL in separate lines.
#@markdown <br>Example URLs:<br>https://www.pornhub.com/model/luxurygirl<br>https://www.pornhub.com/pornstar/leolulu<br>https://www.pornhub.com/channels/mia-khalifa<br>https://www.pornhub.com/users/lasse98<br>https://www.pornhub.com/playlist/30012401
!python3 phdler.py add "$item"

In [ ]:
#@title List
#@markdown Lists videos in database.
item = "model" #@param ["model", "pornstar", "channel", "user", "playlist", "batch"]
!python3 phdler.py list "$item"

In [ ]:
#@title Delete
#@markdown This will list the selected item type, list them from the database and give you an option to enter the item ID of which one you want to be deleted.
item = "model" #@param ["model", "pornstar", "channel", "user", "playlist"]

# Archive/ Unarchive

##__Unzip/Unrar/Untar Files__

###__Unzip Files__



*   First path in below cell - Your Google Drive file location which you want to Unzip
*   Second path in below cell - Path where you want to extract the zip file in Google Drive

In [ ]:
!unzip "/content/drive/MyDrive/John/john-1.9.0-jumbo-1-win64.zip" -d "/content/drive/MyDrive/John/"

###__Unrar Files</center>__





*   First path in below cell - Your Google Drive file location which you want to Unrar
*   Second path in below cell - Path where you want to extract the rar file in Google Drive

*If you want to extract a password protected rar file, then add "-Y" flag or Use this command instead of the command in the cell
(replace the command with the command in the cell)*

* ```
!unrar x -Y "/content/drive/path/input_file_name.rar" "/content/drive/path/output_folder/"
```





In [ ]:
!unrar x "/content/drive/Shareddrives/UNLIMITED220 (2)/Pokemon.rar" "/content/drive/Shareddrives/UNLIMITED220 (2)/Pokemon Sword"

###__Untar Files__



*   First path in below cell - Your Google Drive file location which you want to Untar
*   Second path in below cell - Path where you want to extract the tar file in Google Drive

 *__Flags__*

 * -x : Extract a tar ball.
 * -v : Verbose output or show progress while extracting files.
 * -f : Specify an archive or a tarball filename.
 * -C : Specify a different directory to extract
 * -z : Decompress and extract the contents of the compressed archive created by gzip program (tar.gz extension).
 * -j : Decompress and extract the contents of the compressed archive created by bzip2 program (tar.bz2 extension).

In [ ]:
!tar -xvf "/content/drive/path/input_file_name.tar" -C "/content/drive/path/output_folder/"     #[run this cell to extract tar files]

In [ ]:
!tar -xzvf "/content/drive/path/input_file_name.tar.gz" -C "/content/drive/path/output_folder/"     #[run this cell to extract tar.gz files]

In [ ]:
!tar -xjvf "/content/drive/path/input_file_name.tar.bz2" -C "/content/drive/path/output_folder/"     #[run this cell to extract tar.bz2 files]

###__7zip__

In [ ]:
!apt-get install p7zip-full

In [ ]:
!7z x "/content/drive/MyDrive/John/Hashes.org.7z"

##__Zip/Rar/Tar Files/Folders__

###__Zip Files/Folders__



*   First path in below cell - Your Zip output filename
*   Second path in below cell - Google Drive file location of the files which you want to Zip

In [ ]:
!zip -r "/content/drive/Shareddrives/SharingBackup/Google Photos.zip" "/content/drive/Shareddrives/SharingBackup/TAKEOUT 17.08.2022/Takeout/Google Photos"

###__Rar Files/Folders__





In [ ]:
# <= Click this cell to rar the folder

!apt-get install rar

*   First path in below cell - Your Rar File output Name (do not put .rar at the end. It will take it automatically)
*   Second path in below cell - Google Drive file location of the files/folder which you want to Rar

*If you want to create a password protected rar file, then add "-p" flag or Use this command instead of the command in in the cell.
(replace the command with the command in the cell) After running the cell, it will ask you to provide the password*

* ```
!rar a -p "/content/drive/path/output_file_name" "/content/drive/path/input_file_or_folder"
```

In [ ]:
!rar a "/content/drive/Shareddrives/SharingBackup/Google Photos" "/content/drive/Shareddrives/SharingBackup/TAKEOUT 17.08.2022/Takeout/Google Photos"

###__Tar Files/Folders__



*   First path in below cell - Your Tar output filename
*   Second path in below cell - Google Drive file location of the files which you want to Tar

 *__Flags__*

 * -c : Create a new archive
 * -v : Verbose output
 * -f : Use the given archive file (name.extention)
 * -z : Filter the archive through gzip
 * -j : Filter the archive through bzip2

In [ ]:
!tar -cvf "/content/drive/path/output_file_name.tar" "/content/drive/path/input_file_or_folder"     #[run this cell to create tar files]

In [ ]:
!tar -czvf "/content/drive/path/output_file_name.tar.gz" "/content/drive/path/input_file_or_folder"     #[run this cell to create tar.gz files]

In [ ]:
!tar -cjvf "/content/drive/path/output_file_name.tar.bz2" "/content/drive/path/input_file_or_folder"      #[run this cell to create tar.bz2 files]

# Video Converter

In [ ]:
#@title Install FFmpeg
from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/yunooooo/gcct/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')
os.system('pip install git+git://github.com/AWConant/jikanpy.git')
os.system('add-apt-repository -y ppa:jonathonf/ffmpeg-4')
os.system('apt-get update')
os.system('apt install mediainfo')
os.system('apt-get install ffmpeg')
clear_output()
print('Installation finished.')

## FFmpeg

### <font size="4">» </font><font size="5">Display Media File Metadata

In [ ]:
import os, sys, re

media_file_path = "" #@param {type:"string"}

os.environ['inputFile'] = media_file_path

!ffmpeg -i "$inputFile" -hide_banner

> *You can ignore the* "`At least one output file must be specified`" *error after running this.*




### <font size="4">» </font><font size="5">Convert *Video File* ➔ *.mp4* (*Lossless*)

In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -c copy -strict -2 "$outputPath"/"$fileName".mp4

### <font size="4">» </font><font size="5">Convert *Video File* ➔ *.mkv* (*Lossless*)

In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -c copy -strict -2 "$outputPath"/"$fileName".mkv

### <font size="4">» </font><font size="5">Merge *Video File* + *Subs File* ➔ *.mkv* (*Lossless*)

In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}
subs_file_path = "" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['inputSubsFile'] = subs_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -i "$inputSubsFile" -c:v copy -c:a copy "$outputPath"/"$fileName"-out.mkv

### <font size="4">» </font><font size="5">Trim Video File (*Lossless*)

In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}
start_time = "00:00:00.000" #@param {type:"string"}
end_time = "00:01:00.000" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['startTime'] = start_time
os.environ['endTime'] = end_time
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -ss "$startTime" -to "$endTime" -c copy "$outputPath"/"$fileName"-TRIM."$fileExtension"

### <font size="4">» </font><font size="5">Crop Video

<h3> Crop Variables Explanation:

* `out_width` = The width of your cropped video file.
* `out_height` = The height of your cropped video file.
* `starting_position_x` & `starting_position_y` = These values define the x & y coordinates of the top left corner of your original video to start cropping from.

###### *Example: For cropping the black bars from a video that looked like* [this](https://yuju.pw/y/312r.png):
* *For your starting coordinates* (`x` , `y`) *you would use* (`0` , `138`).
* *For* `out_width` *you would use* `1920`. *And for* `out_height` *you would use `804`.*





In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}
out_width = "1920" #@param {type:"string"}
out_height = "804" #@param {type:"string"}
starting_position_x = "0" #@param {type:"string"}
starting_position_y = "138" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['outWidth'] = out_width
os.environ['outHeight'] = out_height
os.environ['positionX'] = starting_position_x
os.environ['positionY'] = starting_position_y
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -filter:v "crop=$outWidth:$outHeight:$positionX:$positionY" "$outputPath"/"$fileName"-CROP."$fileExtension"

### <font size="4">» </font><font size="5">Extract Audio from Video File (*Lossless*)

In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}
output_file_extension = 'm4a' #@param ["m4a", "mp3", "opus", "flac", "wav"]

delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
output_file_path = re.search("^[\/].+\/", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path.group(0)
os.environ['fileName'] = filename_raw
os.environ['fileType'] = output_file_extension

!ffmpeg -hide_banner -i "$inputFile" -vn -c:a copy "$outputPath"/"$fileName"-audio."$fileType"

### <font size="4">» </font><font size="5">Re-encode a Video to a Different Resolution

In [ ]:
import os, sys, re

video_file_path = '' #@param {type:"string"}
resolution = '1080p' #@param ["2160p", "1440p", "1080p", "720p", "480p", "360p", "240p"]
file_type = 'mp4' #@param ["mkv", "mp4"]

delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
testsplit = video_file_path.split("/")
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
resolution_raw = re.search("[^p]{3,4}", resolution)
output_file_path = re.search("^[\/].+\/", video_file_path)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path.group(0)
os.environ['fileName'] = filename_raw
os.environ['fileType'] = file_type
os.environ['resolutionHeight'] = resolution_raw.group(0)

!ffmpeg -hide_banner -i "$inputFile" -vf "scale=-1:"$resolutionHeight"" -c:a copy -strict experimental "$outputPath"/"$fileName"-"$resolutionHeight"p."$fileType"

### <font size="4">» </font><font size="5">Extract Individual Frames from Video

In [ ]:
#@markdown This will create a folder in the same directory titled "`Extracted Frames`"
import os, sys, re

video_file_path = "/content/drive/Shareddrives/UNLIMITED220 (2)/videoplayback.webm" #@param {type:"string"}
start_time = "00:01:42.000" #@param {type:"string"}
end_time = "00:01:47.000" #@param {type:"string"}
frame_rate = "10" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['startTime'] = start_time
os.environ['endTime'] = end_time
os.environ['frameRate'] = frame_rate
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!mkdir "$outputPath"/"Extracted Frames4"
!ffmpeg -hide_banner -i "$inputFile" -ss "$startTime" -to "$endTime" -r "$frameRate"/1 "$outputPath"/"Extracted Frames4"/frame%04d.png

### <font size="4">» </font><font size="5">Generate Thumbnails - Preview from Video (3x2)

In [ ]:
#@markdown Example of output image: https://yuju.pw/y/39i2.png <br>
import os, sys, re

video_file_path = "" #@param {type:"string"}
output_file_type = 'png' #@param ["png", "jpg"]

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['outputExtension'] = output_file_type
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -vframes 1 -q:v 2 -vf "select=not(mod(n\,200)),scale=-1:480,tile=3x2" -an "$outputPath"/"$fileName"_thumbnails."$outputExtension"

### <font size="4">» </font><font size="5">Convert Audio Filetype (*mp3, m4a, ogg, flac, etc.*)

In [ ]:
import os, sys, re

audio_file_path = "" #@param {type:"string"}
output_file_type = "mp3" #@param ["mp3", "ogg", "m4a", "opus", "flac", "alac", "wav"]

output_file_path = re.search("^[\/].+\/", audio_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", audio_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = audio_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['fileExtension'] = output_file_type
os.environ['fileName'] = filename_raw

!ffmpeg -hide_banner -i "$inputFile" "$outputPath"/"$fileName"converted."$fileExtension"

### <font size="4">» </font><font size="5">Extract + Upload Frames from Video</font>

In [ ]:
import os, re, time, pathlib
import urllib.request
from IPython.display import clear_output

Auto_UP_Gdrive = False
AUTO_MOVE_PATH = "/content"
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'

if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    runSh,
    findProcess,
    loadingAn,
    updateCheck,
    ngrok
)

video_file_path = "/content/drive/Shareddrives/UNLIMITED220 (2)/videoplayback.webm" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!mkdir -p "/content/frames"

for i in range(10):
    clear_output()
    loadingAn()
    print("Uploading Frames...")

%cd "/content/frames"
!ffmpeg -hide_banner -ss 00:56.0 -i "$inputFile" -vframes 1 -q:v 1 -y "/content/frames/frame1.png"
!curl --silent -F "reqtype=fileupload" -F "fileToUpload=@frame1.png" https://catbox.moe/user/api.php -o frame1.txt
f1 = open('frame1.txt', 'r')
%cd "/content"
file_content1 = f1.read()

%cd "/content/frames"
!ffmpeg -hide_banner -ss 02:20.0 -i "$inputFile" -vframes 1 -q:v 1 -y "/content/frames/frame2.png"
!curl --silent -F "reqtype=fileupload" -F "fileToUpload=@frame2.png" https://catbox.moe/user/api.php -o frame2.txt
%cd "/content/frames"
f2 = open('frame2.txt', 'r')
%cd "/content"
file_content2 = f2.read()

clear_output()
print ("Screenshot URLs:")
print ("1. " + file_content1)
print ("2. " + file_content2)

### <font size="4">» </font><font size="5">MediaInfo</font>

In [ ]:
path = "" #@param {type:"string"}
save_txt = False #@param {type:"boolean"}
import os, uuid, re, IPython
import ipywidgets as widgets
import time

from glob import glob
from IPython.display import HTML, clear_output
from google.colab import output, drive

def mediainfo():
  display(HTML("<br>"))
#   print(path.split("/")[::-1][0])
  display(HTML("<br>"))
#   media = !mediainfo "$path"
#   media = "\n".join(media).replace(os.path.dirname(path)+"/", "")
  get_ipython().system_raw("""mediainfo --LogFile="/root/.nfo" "$path" """)
  with open('/root/.nfo', 'r') as file:
    media = file.read()
    media = media.replace(os.path.dirname(path)+"/", "")
  print(media)
  get_ipython().system_raw("rm -f '/root/.nfo'")

  if save_txt:
    txt = path.rpartition('.')[0] + ".txt"
    if os.path.exists(txt):
      get_ipython().system_raw("rm -f '$txt'")
    !curl -s https://pastebin.com/raw/TApKLQfM -o "$txt"
    with open(txt, 'a+') as file:
      file.write("\n\n")
      file.write(media)

while not os.path.exists("/content/drive"):
  try:
    drive.mount("/content/drive")
    clear_output(wait=True)
  except:
    clear_output()

if not os.path.exists("/usr/bin/mediainfo"):
  get_ipython().system_raw("apt-get install mediainfo")

mediainfo()

# Copy, Move, Delete

In [ ]:
Mode = "Delete" #@param ["Copy", "Move", "Delete"]
Source = "/content/drive/MyDrive/Games" #@param {type:"string"}
Destination = "/content" #@param {type:"string"}
if Mode == "Copy":
  !cp -av "$Source" "$Destination"
elif Mode == "Move":
  !mv "$Source" "$Destination"
elif Mode == "Delete":
  !rm -rf "$Source"